In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd

In [ ]:
%run -i one_bridge_geometry.ipynb

In [ ]:
bridge_component = input() #arch, pier, foundation, no matter lower/upper case
print(bridge_component)

In [ ]:
#https://www.freecodecamp.org/news/python-switch-statement-switch-case-example/
#https://www.geeksforgeeks.org/check-multiple-conditions-in-if-statement-python/
if bridge_component == 'pier':
    %store -r pier_capacity_geometry_variables
    capacity_geometry_variables = pier_capacity_geometry_variables
elif bridge_component == 'arch':
    %store -r arch_capacity_geometry_variables
    capacity_geometry_variables = arch_capacity_geometry_variables


In [ ]:
print(capacity_geometry_variables)

In [ ]:
#https://www.geeksforgeeks.org/python-string-concatenation/
#https://www.datacamp.com/tutorial/case-conversion-python
b=capacity_geometry_variables["".join(["b","_",bridge_component.lower()])] #https://www.w3schools.com/python/python_dictionaries_access.asp
t=capacity_geometry_variables["".join(["t","_",bridge_component.lower()])]
e=capacity_geometry_variables["".join(["e","_",bridge_component.lower()])]

In [ ]:
#if arch masonry = pier masonry, then masonry_compressive_resistance_realisations are the same
#in the latter, if arch_capacity has already run, there is no need to run the below statement
# %run -i three_compressive_resistance_sampling.ipynb

In [ ]:
%store -r compressive_resistance_realisations

In [ ]:
Rc=compressive_resistance_realisations
# type(Rc)

In [ ]:
# # Rc = compressive_resistance_realisations
# Rc = np.random.normal(100, 20, 100_000) #to run the code with fake Rc realisations during implementation

In [ ]:
%run -i fifteen_pier_condition_factor.ipynb

In [ ]:
%store

In [ ]:
%store -r condition_factor
%store -r pier_local_scour_depth

In [ ]:
deterministic_coefficient = 0.6

In [ ]:
pier_compressive_capacity = []

In [ ]:
def determine_compressive_capacity():

    for i in range(len(condition_factor[0])):
        compressive_capacity = deterministic_coefficient * condition_factor[0][i] * (Rc*1000*b*(t-2*e)) #https://en.wikipedia.org/wiki/Normal_distribution#Operations_on_a_single_normal_variable
        pier_compressive_capacity.append(compressive_capacity)
    
    print(pier_compressive_capacity[0:5])

    return pier_compressive_capacity

In [ ]:
determine_compressive_capacity()

In [ ]:
df_pier_compressive_capacity = pd.DataFrame(pier_compressive_capacity)
#https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-lists/
#https://stackoverflow.com/questions/48864746/replicating-excel-calculation-in-python-dataframe

In [ ]:
df_pier_compressive_capacity.insert(0, "scour depth", pier_local_scour_depth[0], True) 
df_pier_compressive_capacity.insert(1, "condition factor", condition_factor[0], True) 

In [ ]:
df_pier_compressive_capacity

In [ ]:
%store df_pier_compressive_capacity

In [ ]:
#to check if df_arch_compressive_capacity is stored

In [ ]:
%store 

In [ ]:
pier_compressive_capacity_mean = [] #calculation of mean value of capacity realisations for each investigated condition factor

In [ ]:
pier_compressive_capacity_vector = np.array(pier_compressive_capacity)  

Example of capacity pdf

In [ ]:
def pier_compressive_capacity_mean_calculation():
    
    for i in range(len(pier_compressive_capacity_vector)):
        pier_compressive_capacity_vector_mean = np.mean(pier_compressive_capacity_vector[i])
        pier_compressive_capacity_mean.append(pier_compressive_capacity_vector_mean)
    
    return pier_compressive_capacity_mean

In [ ]:
pier_compressive_capacity_mean_calculation()

In [ ]:
pier_compressive_capacity_std=[] #calculation of std value of capacity realisations for each investigated condition factor

In [ ]:
def pier_compressive_capacity_std_calculation():
    
    for i in range(len(pier_compressive_capacity_vector)):
        pier_compressive_capacity_vector_std = np.std(pier_compressive_capacity_vector[i])
        pier_compressive_capacity_std.append(pier_compressive_capacity_vector_std)
    
    return pier_compressive_capacity_std

In [ ]:
pier_compressive_capacity_std_calculation()

In [ ]:
example_indexes = (np.array(pier_local_scour_depth) > 1.00).nonzero() #https://stackoverflow.com/questions/16094563/numpy-get-index-where-value-is-true
first_example_indexes = example_indexes[1][0]

In [ ]:
pier_compressive_capacity_pdf_input = np.linspace(pier_compressive_capacity_mean[first_example_indexes]-3*pier_compressive_capacity_std[first_example_indexes], pier_compressive_capacity_mean[first_example_indexes]+3*pier_compressive_capacity_std[first_example_indexes], len(Rc))

In [ ]:
pier_compressive_capacity_pdf = scipy.stats.norm.pdf(pier_compressive_capacity_pdf_input, pier_compressive_capacity_mean[first_example_indexes], pier_compressive_capacity_std[first_example_indexes])

In [ ]:
plt.plot(pier_compressive_capacity_pdf_input, pier_compressive_capacity_pdf, color='black')
plt.xlabel("Compressive resistance force")
plt.ylabel("Probability density")